MSU-USDA Python Workshop

# Python for Data Science: Sample Project

## Titanic Dataset

A commonly used first machine learning project is the Titanic data challenge. The goal is to classify which passengers will survive based on their characteristics. We'll use the dataset in this sample project to learn some of the basics of working with tabular data. The ML part will just be for fun at the end.

## I. Importing Libraries

First, let's import the libraries needed both for data wrangling, for visualization, and for a little machine learning.

In [ ]:
# Import the necessary libraries for Data Wrangling

import pandas as pd
import numpy as np

# Import some libraries for machine learning
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree

# Import some libraries for visualization
# Seaborn is really nice, as it works alongside matplotlib, and it is beautiful.
# Seaborn documentation is here: https://seaborn.pydata.org/
import seaborn as sns
import matplotlib.pyplot as plt


## II. Exploring the Data

### Loading Data from a Local File

In [ ]:
# load your passenger data
# note how you can tell pandas to treat an individual column as a number type
# right from the beginning - here we're telling it to treat the 'Age' column
# as a number.
passengers = pd.read_csv("data/titanic.csv",dtype={'Age': np.float64})

# Windows users - you may need to do a few additional things to make this read_csv work.
# Windows requires an "r" before the file path. Something like: r'C:\Users\username\titanic.csv'

# Then, let's describe the data just to be sure our read_csv worked
passengers.describe()

### Exercise 1: Viewing Head and Tail

In [ ]:
# Exercise 1:
    
# a. Supply the code necessary to view six lines of the data's head


In [ ]:
# b. Supply the code necessary to view six lines data's tail

# c. What do you notice about the last line? 
# Put Answer here: 

### Dealing with a Null Row

Yes, the last line is a problem. Sometimes this happens if your CSV file has a line return at the end. We should definitely drop it.

An easy way to handle this is to simply drop the rows where all values are null.

In [ ]:
passengers = passengers.dropna(how='all')
passengers

### Features of the Dataset

The Features of this Dataset Include:

    survival - Survival (0 = No; 1 = Yes)
    class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
    name - Name
    sex - Sex
    age - Age
    sibsp - Number of Siblings/Spouses Aboard
    parch - Number of Parents/Children Aboard
    ticket - Ticket Number
    fare - Passenger Fare
    cabin - Cabin
    embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
    boat - Lifeboat (if survived)
    body - Body number (if did not survive and body was recovered)

"Sex" is an interesting data field/column/property. Think of the movie 'Titanic' and of the patterns of etiquette during danger that might have prevailed in 1912. I wonder what percentage of women vs. men died?

### Example: Finding the Survival Rate for Women

In [ ]:
# Get a list of "survived" by limiting to places where sex is "female"
# non-query ways to do the same thing:  
# women = passengers.loc[passengers.sex == 'female']
# or
# women = passengers.loc[passengers['sex'] == 'female']
women = passengers.query("sex == 'female'")

# To get the survival rate, divide the sum of survivals (add up all the 1's) 
# and divide by the number of women

rate_women = sum(women['survived'])/len(women['survived'])

print("% of women who survived:", rate_women)


Wow, 73% of women passengers survived. What about the percentage of men? That's your exercise.

### Exercise 2: Finding the Survival Rate for Men

In [ ]:
# Exercise 2:

# a. find a list containing survival info for men

# b. print the data type that was returned by the previous operation:

# c. calculate the survival rate of men

# d. print out the survival rate for men

# e. what do you notice about men's vs. women's survival rates?
# put your answer here in a comment:
# 

We're starting to understand this data a little bit.

But maybe we should have visualized the data first and may have seen this sex difference right away.

## III. Visualizing the Data

Let's use the seaborn "pair plot" to look at the relationship between each variable in the dataset and survival.

You should take this opportunity to read about the pair plot (https://pythonbasics.org/seaborn-pairplot/) and how it can be used in exploratory analysis.

Run the following code and be amazed.

In [ ]:
# call the pairplot and shade the values based on survival
# note, this code takes a few seconds to run
sns.pairplot(data=passengers, hue='survived')
plt.show()

Take a minute to figure out what this grid means - that each row contains pair plots of each property to every other property.

### Exercise 3: Questions about the Pair Plot

In [ ]:
# Exercise 3

# a. Question: Why are there no survivors shown on the "body" row?
# put answer here in a comment
# 

# b. Question: why do you think that the sex columns wasn't visualized?
# put answer here in a comment
# 

# c. Using the documentation of seaborn pair plots, show only the plots only for class, age, and fare price.


## IV. Handling Missing Values

Now, let's deal with missing values. 

### Seeing What's Missing

First, we'll see what's missing.

In [ ]:
# Are there missing values?

passengers.isnull().sum()

Start with the easiest first. Just two rows are missing the location from which the passenger embarked. Do we really think the place of embarking would make a difference in whether the person lived or died? Are Cherbourg, Queenstown, and Southampton radically different in terms of the strength of swimmers who are born there?

### Replacing Null Values with a Placeholder

So, let's just fill in any unknowns with a new "unknown" town and be done with it.

In [ ]:
# remember .loc[rows,columns]
# find the rows where embarked is null, and assign 'U' to the 'embarked' column on those rows:
passengers.loc[passengers['embarked'].isnull(), 'embarked'] = 'U'

passengers.isnull().sum()

Great, now we have no nulls in the embarked series.

### Example: Replacing Missing Numerical Values with a Median Value

Now, let's tackle age and fare. We're missing 263 values and 1 value respectively there. How about we just replace unknown ages with the median age value and unknown fares with the median fare value? It will depend upon your subject matter and dataset whether you prefer to replace with median or mean.

In [ ]:
# Replace missing ages with the median age. Notice that we're using 
# convenient "dot notation" to reference columns
passengers.loc[passengers.age.isnull(), 'age'] = passengers.age.median()

passengers.isnull().sum()

### Exercise 4: Replacing Missing Fare with Median Fare

In [ ]:
# Exercise 4

# a. Replace missing fare values with the median fare

# b. Show a summary of null fields to see where we now stand in cleaning our data


### Exercise 5: Replacing Missing Cabin with Unknown

Now, the "cabin" column looks a little freeform. How about we create an Unknown Cabin entry into any blank cabin fields? Maybe the fact that the cabin is "unknown" is a good piece of information - maybe these are stowaways or vagabonds?

In [ ]:
# Exercise 5

# a. replace the missing cabins with the string "Unknown"

# b. Show a summary of null fields to see where we stand


Since "body" and "boat" essentially only exist if the person's body was tagged (deceased) or if they were saved in a boat (survived), then we don't really need to worry about those columns. They essentially just mean the same thing as the "survived" column and we aren't likely to use them.

"Destination" also feels like something that wouldn't matter to survival, so let's not even worry about that feature. In real life, you'd make your decision about how to wrangle these columns based on your goals for the analysis.

## V. Handling Categorical Data

Some features are categorical, though, and we need to make them numeric for machine learning.

### Example: Creating Dummy Variables for Sex

We've already seen that sex is probably determinative in some cases for survival, so let's convert sex from a categorical variable into two dummy variables (one-hot encoded variables) using the Pandas function [df.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

Machine learning practitioners call this process of making new columns for analysis, "feature engineering."

In [ ]:
# This code will create dummy variables from a single categorical column
passengers = pd.get_dummies(passengers, columns=["sex"], prefix="sex")
passengers



### Exercise 6: Creating Dummy Variables for Embarkation

In [ ]:
# Exercise 6

# a. Which columns were added? 
# put answer in a comment
# 

# b. Why were the columns named as they are?
# put answer in a comment
# 

# c. Create dummy variables for the "embarked" and use "leftFrom" as the prefix for the name

# d. show the resulting data


## VI. Converting a Number to Boolean or Int

Survived still doesn't look like a boolean column. Pandas probably just treated the number like a floating point number instead of a 1/0 yes/no.

In [ ]:
# Let's interpret that column as a plain int since Boolean would create true/false and we 
# like to use numbers for machine learning
# Notice here that we're using 'dot syntax' - I could have used passengers['survived'] instead of
# passengers.survived
# Note, with astype you could use str, numpy.dtype, pandas.ExtensionDtype or any Python type
passengers.survived=passengers.survived.astype(int)

passengers

## VII. Data are Beautiful, Now Let's Split for Machine Learning

Now, it's time for some magic - creating the data we need for machine learning.

Please don't worry if there is a lot of new stuff in the next few cells. Just go along for the ride and try to understand each part.

We will want to divide our dataset into training data to build a model and then test data that will allow us to test our model on new data that we didn't use to cheat build our model.

### Get our Labels
Machine Learning naming conventions are to use lower case "y" for our data labels that we'll use during training.

In [ ]:
# Get our survival training labels in a Pandas series
y = passengers.survived

### Get our Features to use for Training

Now, we can get our data that we want to include for training. These would be the fields that we think might be relevant to predicting whether a passenger survived (y=1) or perished (y=0).

Let's start with class, age, and sibsp only.

The naming convention for the data we'll be using for training is upper-case X ("X")

In [ ]:
# make a list of our features
analysis_features = ['pclass','age','sibsp']

# grab a data frame that contains our features
X = passengers[analysis_features]

# let's see what it looks like
X

### Split the data into training and test data.

Then, let's be sure we create training / test data using "train_test_split" - something we imported from scikit learn. The documentation is here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# Notice how cool it is that a method in Python can return more than one value!
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 1)

# For example, let's look at our validation labels
val_y

## VIII. Training Machine Learning Models

Now, let's try to do some model training using a simple machine learning algorithm known as "decision trees."

### Train the Model

We'll use a simple "classifier" since our goal is just to predict the bare yes/no - did the passenger die?

In [ ]:
# Specify Model
ourTreeModel1 =  DecisionTreeClassifier(random_state=1)

# Fit Model - the .fit method tries to fit the model to the training data set (X) against the labels (y)
ourTreeModel1.fit(train_X, train_y)


### Validate the Model

Now, let's validate our new model against test data

In [ ]:
# first, get predictions on the test data features
survivalPredictions = ourTreeModel1.predict(val_X)

# calculate accuracy score (percentage of correct classifications)
accuracyScore = accuracy_score(val_y,survivalPredictions)

print(f"Validation Accuracy: {accuracyScore}")

Wow, 62.5% accuracy. Better than a coin flip, but not better than if we simply guess based on sex.

### Exercise 7: Adding More Features to our Decision Tree

In [ ]:
# Exercise 7: Redo with More Features

# a. For this exercise, re-run the split, training, and validation 
# but before splitting, start with a dataset that also includes our two sex columns

# b. Is your accuracy better with the sex features added?
# 

### Exercise 8: Using a Random Forest Method

In [ ]:
# Exercise 8: Redo with Different Method

# a. For this exercise, re-run the training, and validation
# but use a Random Forest Classifier with these settings
# ourModel = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)


# b. Is your accuracy better with a better technique?
# 

### Bonus Exercise: For Early Finishers

In [ ]:
# Bonus: If you have time

# Re-run the training, and validation
# but use a different classifier from scikit like
# Nearest Neighbors, Linear SVM, RBF SVM, Gaussian Process, Neural Net, AdaBoost, Naive Bayes, QDA, etc.



## IX. For Further Learning

This sample project has introduced you to the basics of loading data, cleaning it, feature engineering, and running a basic ML model. You've learned more than you need to work with the Kaggle [Titanic Competition](https://www.kaggle.com/competitions/titanic). This project at Kaggle introduces you to the world of competitive machine learning where you can participate or just observe experts solving big problems with Python and various machine learning or AI libraries.